In [1]:
import requests
import pandas as pd
from config import weather_api_key

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [3]:
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
          "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
          "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
          "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
          "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
          "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

abbr = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
       "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
       "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]


cities = ["Birmingham", "Anchorage", "Phoenix", "Little Rock", "Los Angeles", "Denver", "Bridgeport", "Wilmington", "Jacksonville",
         "Atlanta", "Honolulu", "Boise", "Chicago", "Indianapolis", "Des Moines", "Wichita", "Louisville", "New Orleans", "Portland",
         "Baltimore", "Boston", "Detroit", "Minneapolis", "Jackson", "Kansas City", "Billings", "Omaha", "Las Vegas", 
         "Manchester", "Newark", "Albuquerque", "New York City", "Charlotte", "Fargo", "Columbus", "Oklahoma City", "Portland",
         "Philadelphia", "Providence", "Charleston", "Sioux Falls", "Nashville", "Houston", "Salt Lake City", "Burlington",
         "Virginia Beach", "Seattle", "Charleston", "Milwaukee", "Cheyenne"]

temp = []
dic = []

for city in cities:
    response = requests.get(query_url + city).json()
    a = {
        "city": city,
        "temp": response['main']['temp']
    }
    dic.append(a)
    temp.append(response['main']['temp'])

In [4]:
weather_data = pd.DataFrame(dic)
weather_data.head()

,city,temp
0,Birmingham,63.19
1,Anchorage,14.68
2,Phoenix,71.98
3,Little Rock,59.00
4,Los Angeles,76.78


In [5]:
weather_data['state'] = states
weather_data['abbr'] = abbr

In [6]:
weather_data.head()

,city,temp,state,abbr
0,Birmingham,63.19,Alabama,AL
1,Anchorage,14.68,Alaska,AK
2,Phoenix,71.98,Arizona,AZ
3,Little Rock,59.00,Arkansas,AR
4,Los Angeles,76.78,California,CA


In [7]:
weather_df = weather_data[['state', 'abbr', 'city', 'temp']]

In [8]:
weather_df.head()

,state,abbr,city,temp
0,Alabama,AL,Birmingham,63.19
1,Alaska,AK,Anchorage,14.68
2,Arizona,AZ,Phoenix,71.98
3,Arkansas,AR,Little Rock,59.00
4,California,CA,Los Angeles,76.78


In [9]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import lxml.html as lh

In [10]:
#setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Robert Anthony\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [11]:
url = "https://dqydj.com/average-income-by-state-median-top-percentiles/"
browser.visit(url)
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [12]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [13]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 State
2 Average Income


In [14]:
for j in range(53, 104):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 4, the //tr data is not from our table 
    if len(T)!=2:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [15]:
[len(C) for (title,C) in col]

[51, 51]

In [16]:
Dict={title:column for (title,column) in col}
median_df=pd.DataFrame(Dict)

In [17]:
median_df.head()

,State,Average Income
0,Alabama,"$55,935.00"
1,Alaska,"$78,038.00"
2,Arizona,"$70,080.00"
3,Arkansas,"$54,322.00"
4,California,"$78,002.00"


In [18]:
median_df = median_df.rename(columns={"State": "state", "Average Income": "median income"})

In [19]:
median_df = median_df.drop(median_df.index[[8]])

In [20]:
merged_df = pd.merge(left=weather_df, right=median_df, left_on='state', right_on='state')

In [21]:
merged_df.head()

,state,abbr,city,temp,median income
0,Alabama,AL,Birmingham,63.19,"$55,935.00"
1,Alaska,AK,Anchorage,14.68,"$78,038.00"
2,Arizona,AZ,Phoenix,71.98,"$70,080.00"
3,Arkansas,AR,Little Rock,59.00,"$54,322.00"
4,California,CA,Los Angeles,76.78,"$78,002.00"


In [22]:
#poverty csv
poverty_df = pd.read_csv("../Data/Rob/State_Poverty.csv")

In [23]:
poverty_df.head()

,State,PovertyRate,undefined
0,Mississippi,0.1975,585786
1,Louisiana,0.1883,871467
2,New Mexico,0.1863,392065
3,West Virginia,0.1754,310044
4,Kentucky,0.1667,747010


In [24]:
poverty_df = poverty_df.drop(columns=['undefined'])

In [25]:
poverty_df = poverty_df.rename(columns = {'State': 'state', 'PovertyRate': 'poverty'})

In [26]:
poverty_df.head()

,state,poverty
0,Mississippi,0.1975
1,Louisiana,0.1883
2,New Mexico,0.1863
3,West Virginia,0.1754
4,Kentucky,0.1667


In [27]:
poverty_df = poverty_df.sort_values('state')

In [28]:
poverty_df = poverty_df.reset_index(drop = True)

In [29]:
poverty_df["poverty"] = 100 * poverty_df["poverty"]

In [30]:
poverty_df.head()

,state,poverty
0,Alabama,16.13
1,Alaska,10.62
2,Arizona,13.88
3,Arkansas,16.36
4,California,13.00


In [31]:
merged_df_2 = pd.merge(left=merged_df, right=poverty_df, left_on='state', right_on='state')

In [32]:
merged_df_2.head()

,state,abbr,city,temp,median income,poverty
0,Alabama,AL,Birmingham,63.19,"$55,935.00",16.13
1,Alaska,AK,Anchorage,14.68,"$78,038.00",10.62
2,Arizona,AZ,Phoenix,71.98,"$70,080.00",13.88
3,Arkansas,AR,Little Rock,59.00,"$54,322.00",16.36
4,California,CA,Los Angeles,76.78,"$78,002.00",13.00


In [33]:
merged_df_2 = merged_df_2.replace(',', '', regex=True)

In [34]:
merged_df_2['median income'] = merged_df_2['median income'].str.replace('$', '')

In [35]:
merged_df_2.head()

,state,abbr,city,temp,median income,poverty
0,Alabama,AL,Birmingham,63.19,55935.00,16.13
1,Alaska,AK,Anchorage,14.68,78038.00,10.62
2,Arizona,AZ,Phoenix,71.98,70080.00,13.88
3,Arkansas,AR,Little Rock,59.00,54322.00,16.36
4,California,CA,Los Angeles,76.78,78002.00,13.00


In [36]:
merged_df_2['median income'] = merged_df_2['median income'].astype(float)

In [37]:
merged_df_2 = merged_df_2.round(1)

In [38]:
merged_df_2.head()

,state,abbr,city,temp,median income,poverty
0,Alabama,AL,Birmingham,63.2,55935.0,16.1
1,Alaska,AK,Anchorage,14.7,78038.0,10.6
2,Arizona,AZ,Phoenix,72.0,70080.0,13.9
3,Arkansas,AR,Little Rock,59.0,54322.0,16.4
4,California,CA,Los Angeles,76.8,78002.0,13.0


In [39]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import lxml.html as lh

In [40]:
#setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Robert Anthony\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [41]:
#Mean travel time to work (minutes), workers age 16 years+, 2014-2018
url = "https://www.indexmundi.com/facts/united-states/quick-facts/all-states/average-commute-time#table"
browser.visit(url)
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [42]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [43]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 State
2 Value


In [44]:
for j in range(0, 52):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 4, the //tr data is not from our table 
    if len(T)!=2:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [45]:
[len(C) for (title,C) in col]

[52, 52]

In [46]:
Dict={title:column for (title,column) in col}
commute_df=pd.DataFrame(Dict)

In [47]:
commute_df = commute_df.drop([0, 9])

In [48]:
commute_df = commute_df.reset_index(drop=True)

In [49]:
commute_df = commute_df.rename(columns={"State": "state", "Value": "commute"})

In [50]:
commute_df.head()

,state,commute
0,Alabama,24.7
1,Alaska,18.8
2,Arizona,25.3
3,Arkansas,21.7
4,California,29.3


In [51]:
merged_df_3 = pd.merge(left=merged_df_2, right=commute_df, left_on='state', right_on='state')

In [52]:
merged_df_3.head()

,state,abbr,city,temp,median income,poverty,commute
0,Alabama,AL,Birmingham,63.2,55935.0,16.1,24.7
1,Alaska,AK,Anchorage,14.7,78038.0,10.6,18.8
2,Arizona,AZ,Phoenix,72.0,70080.0,13.9,25.3
3,Arkansas,AR,Little Rock,59.0,54322.0,16.4,21.7
4,California,CA,Los Angeles,76.8,78002.0,13.0,29.3


In [53]:
#health_rankings csv
health_df = pd.read_csv("../Data/Rob/Health_Rankings.csv")

In [54]:
health_df = health_df.drop(columns=['RANK'])

In [55]:
health_df = health_df.rename(columns={'STATE': 'state', 'VALUE': 'health'})

In [56]:
health_df.head()

,state,health
0,Alabama,-0.838
1,Alaska,-0.115
2,Arizona,-0.105
3,Arkansas,-0.672
4,California,0.431


In [57]:
merged_df_4 = pd.merge(left=merged_df_3, right=health_df, left_on='state', right_on='state')

In [58]:
merged_df_4.head()

,state,abbr,city,temp,median income,poverty,commute,health
0,Alabama,AL,Birmingham,63.2,55935.0,16.1,24.7,-0.838
1,Alaska,AK,Anchorage,14.7,78038.0,10.6,18.8,-0.115
2,Arizona,AZ,Phoenix,72.0,70080.0,13.9,25.3,-0.105
3,Arkansas,AR,Little Rock,59.0,54322.0,16.4,21.7,-0.672
4,California,CA,Los Angeles,76.8,78002.0,13.0,29.3,0.431


In [59]:
merged_df_4 = merged_df_4.round(3)

In [60]:
merged_df_4 = merged_df_4.rename(columns={'median income': 'm_income'})

In [61]:
merged_df_4.head()

,state,abbr,city,temp,m_income,poverty,commute,health
0,Alabama,AL,Birmingham,63.2,55935.0,16.1,24.7,-0.838
1,Alaska,AK,Anchorage,14.7,78038.0,10.6,18.8,-0.115
2,Arizona,AZ,Phoenix,72.0,70080.0,13.9,25.3,-0.105
3,Arkansas,AR,Little Rock,59.0,54322.0,16.4,21.7,-0.672
4,California,CA,Los Angeles,76.8,78002.0,13.0,29.3,0.431


In [62]:
health_list = merged_df_4['health'].tolist()

In [63]:
#normalize neg & pos values in health_list to range(0, 2)
old_min = min(health_list)
old_range = max(health_list) - old_min

In [64]:
new_min = 1
new_range = 1.2 + 0.9999999999 - new_min
new_health =[]
for n in health_list:
    output = ((n - old_min) / old_range * new_range + new_min)
    new_health.append(output)

In [65]:
merged_df_4['new_health'] = new_health

In [66]:
merged_df_4 = merged_df_4.rename(columns={'health': 'old_health', 'new_health': 'health'})

In [67]:
merged_df_4 = merged_df_4.round(3)

In [68]:
merged_df_4.head()

,state,abbr,city,temp,m_income,poverty,commute,old_health,health
0,Alabama,AL,Birmingham,63.2,55935.0,16.1,24.7,-0.838,1.115
1,Alaska,AK,Anchorage,14.7,78038.0,10.6,18.8,-0.115,1.571
2,Arizona,AZ,Phoenix,72.0,70080.0,13.9,25.3,-0.105,1.578
3,Arkansas,AR,Little Rock,59.0,54322.0,16.4,21.7,-0.672,1.220
4,California,CA,Los Angeles,76.8,78002.0,13.0,29.3,0.431,1.916


In [74]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import lxml.html as lh

In [75]:
#setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Robert Anthony\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [76]:
#Mean travel time to work (minutes), workers age 16 years+, 2014-2018
url = "https://www.indexmundi.com/facts/united-states/quick-facts/all-states/homeownership-rate#table"
browser.visit(url)
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [77]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [78]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 State
2 Value


In [79]:
for j in range(0, 52):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 4, the //tr data is not from our table 
    if len(T)!=2:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [80]:
[len(C) for (title,C) in col]

[52, 52]

In [81]:
Dict={title:column for (title,column) in col}
ownership_df=pd.DataFrame(Dict)

In [83]:
ownership_df.head()

,State,Value
0,State,Value
1,Alabama,68.6
2,Alaska,64.0
3,Arizona,63.6
4,Arkansas,65.7


In [84]:
ownership_df = ownership_df.drop([0, 9])

In [85]:
ownership_df = ownership_df.reset_index(drop=True)

In [86]:
ownership_df = ownership_df.rename(columns={"State": "state", "Value": "ownership"})

In [88]:
ownership_df.head()

,state,ownership
0,Alabama,68.6
1,Alaska,64.0
2,Arizona,63.6
3,Arkansas,65.7
4,California,54.6


In [89]:
merged_df_5 = pd.merge(left=merged_df_4, right=ownership_df, left_on='state', right_on='state')

In [92]:
merged_df_5.head()

,state,abbr,city,temp,m_income,poverty,commute,old_health,health,ownership
0,Alabama,AL,Birmingham,63.2,55935.0,16.1,24.7,-0.838,1.115,68.6
1,Alaska,AK,Anchorage,14.7,78038.0,10.6,18.8,-0.115,1.571,64.0
2,Arizona,AZ,Phoenix,72.0,70080.0,13.9,25.3,-0.105,1.578,63.6
3,Arkansas,AR,Little Rock,59.0,54322.0,16.4,21.7,-0.672,1.220,65.7
4,California,CA,Los Angeles,76.8,78002.0,13.0,29.3,0.431,1.916,54.6


In [93]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import lxml.html as lh

In [94]:
#setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Robert Anthony\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [95]:
#Mean travel time to work (minutes), workers age 16 years+, 2014-2018
url = "https://www.currentresults.com/Weather/US/average-annual-state-precipitation.php"
browser.visit(url)
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [108]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

In [109]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 State
2 Inches
3 Milli­metres
4 Rank


In [110]:
for j in range(0, 53):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 4, the //tr data is not from our table 
    if len(T)!=4:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [111]:
[len(C) for (title,C) in col]

[53, 53, 53, 53]

In [112]:
Dict={title:column for (title,column) in col}
precipitation_df=pd.DataFrame(Dict)

In [114]:
precipitation_df.head()

,State,Inches,Milli­metres,Rank
0,State,Inches,Milli­metres,Rank
1,Alabama,58.3,1480,4
2,Alaska,22.5,572,39
3,Arizona,13.6,345,47
4,Arkansas,50.6,1284,8


In [115]:
precipitation_df = precipitation_df.drop([0, 18, 36])

In [116]:
precipitation_df = precipitation_df.reset_index(drop=True)

In [117]:
precipitation_df = precipitation_df.rename(columns={"State": "state", "Inches": "precip"})

In [119]:
precipitation_df = precipitation_df.drop(columns=['Milli­metres', 'Rank'])

In [121]:
precipitation_df.head()

,state,precip
0,Alabama,58.3
1,Alaska,22.5
2,Arizona,13.6
3,Arkansas,50.6
4,California,22.2


In [122]:
merged_df_6 = pd.merge(left=merged_df_5, right=precipitation_df, left_on='state', right_on='state')

In [125]:
merged_df_6.head()

,state,abbr,city,temp,m_income,poverty,commute,old_health,health,ownership,precip
0,Alabama,AL,Birmingham,63.2,55935.0,16.1,24.7,-0.838,1.115,68.6,58.3
1,Alaska,AK,Anchorage,14.7,78038.0,10.6,18.8,-0.115,1.571,64.0,22.5
2,Arizona,AZ,Phoenix,72.0,70080.0,13.9,25.3,-0.105,1.578,63.6,13.6
3,Arkansas,AR,Little Rock,59.0,54322.0,16.4,21.7,-0.672,1.220,65.7,50.6
4,California,CA,Los Angeles,76.8,78002.0,13.0,29.3,0.431,1.916,54.6,22.2


In [126]:
merged_df_6.to_csv('../Data/Rob/Final_Data.csv',index=False)